In [1]:
import ee
import folium
import numpy as np
import pandas as pd
import proplot as plot
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

service_account = 'nattawat@geeproject-313808.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'key.json')
ee.Initialize(credentials)

GEE and the API are new to me. It not Working with API and I can do just select date to download zip file (True color & False color ) and API part I do it can donwload zip file, but GEE and API is not connecting . 

In [2]:
from geojson import Polygon
 
# pp = [[ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0],[100.0, 1.0], [100.0, 0.0] ]]

#I can do just select date to download zip file (True color & False color )
coor  = [[101.2142944, 15.0590284],[101.2407303, 15.0565419],[101.2422752, 15.0787537],[101.2151527, 15.0805769],[101.2142944, 15.0590284]]
polygon = ee.Geometry.Polygon(coor)

mask = 0

F_date = input("Frist Date : ") #2018-01-01
L_date = input("Last Date : ") #2018-01-13 


##############################################
def Download_IMGAE(F_date,polygon):

    def maskSen2_clouds(image):
        qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0)
        mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

        return image.updateMask(mask).divide(10000)

    ###################################
    SATELLITE_ST = 'COPERNICUS/S2'
    first_date = F_date
    last_date = L_date

    print('Polygon GeoJSON: ', polygon.toGeoJSONString()) # Print the geometry as a GeoJSON string.
    print('Geometry type: ', polygon.type().getInfo()) # Print the GeoJSON 'type'

    # Call collection of satellite images.
    collection = (ee.ImageCollection(SATELLITE_ST)
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
                 .select(['B8','B4', 'B3', 'B2','QA60'])
              # Filter for images within a given date range.
                 .filter(ee.Filter.date(first_date,last_date))
              # Filter for images that overlap with the assigned geometry.
                .filterBounds(polygon)
              # Filter for images that have less then 20% cloud coverage.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskSen2_clouds))
              
    image = collection.sort('system:index', opt_ascending=False).mosaic()
    image_RGB = image.visualize( bands=['B4', 'B3', 'B2'],#True color
                                 min=[0.0, 0.0, 0.0],
                                 max=[0.3,0.3, 0.3] )

    image_NRG = image.visualize( bands=['B8', 'B4', 'B3'], #False color
                                 min=[0.0, 0.0, 0.0],
                                 max=[0.3,0.3, 0.3] )        
             
    task_config = {
        'fileFormat': 'GeoTIFF',
        'region': polygon.coordinates().getInfo(),
        'folder': 'Example_Folder_Name',
        'scale': 10,
        'crs': 'EPSG:4326',
        'description': 'Example_File_Name'}

    link_True = image_RGB.getDownloadUrl(task_config)
    link_False = image_NRG.getDownloadUrl(task_config)

    # display('Coppy URL to download True : ', link_True)
    # display('Coppy URL to download False : ', link_False)

    return print('Coppy URL to download True : ', link_True),print('Coppy URL to download False : ', link_False)

Download_IMGAE(F_date,polygon)

Frist Date : 2018-01-01
Last Date : 2018-01-13
Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[101.2142944, 15.0590284], [101.2407303, 15.0565419], [101.2422752, 15.0787537], [101.2151527, 15.0805769], [101.2142944, 15.0590284]]], "evenOdd": true}
Geometry type:  Polygon
Coppy URL to download True :  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6f2daeb96c7d433c58a361074c4c6d64-5b2b93e5b07799e264fe9c3f7eec7f1c:getPixels
Coppy URL to download False :  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5513c069b7eecfe046d777c4fa9e2b56-5c8a0c2de4afe21d390f3b3dc6e115b6:getPixels


(None, None)

In [3]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.FeatureCollection(ee_image_object).getMapId(vis_params) #set 'ee.FeatureCollection','ee.Image','ee.ImageCollection'
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filterMetadata('country_co', 'equals', 'TH')
Map_TH = folium.Map(location=[15.0, 100.5], zoom_start=5.5)
Map_TH.add_ee_layer(thailand,'','THAILAND')
display(Map_TH)